# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import re
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd

# Collect gt annotations and predicted images

In [ ]:
def extract_id(gt_annotation):
  match = re.search(r"_([0-9]+)\.png$", gt_annotation["filename"])

  if match:
      gt_id = match.group(1)

  return gt_id

def find_gt_annotation(gt_annotations,pred_image):
  gt_image = None
  # Returns the gt_annotation that matches the pred_image
  for annotation in gt_annotations:
    if str(extract_id(annotation)) == str(pred_image["image_id"]):
      gt_image = annotation
  return gt_image

def get_pred_image_boxes(pred_image):
  boxes = []
  i=1 #Just if there is more than 1 mouse
  for box in pred_image["boxes"]:
    height= abs(box[1]-box[3])
    width = abs(box[0]-box[2])
    boxes.append({'mouse'+str(i): 'boundingBox',
          'height': height,
          'width': width,
          'x': width/2 + min(box[0],box[2]),
          'y': height/2+min(box[1],box[3])})
    i+=1
  return boxes

# Calculate IoU (Intersection over Union) scores

In [ ]:
def calculate_iou(boxA, boxB):
    # Get coordinates of intersection rectangle
    xA = max(boxA['x'], boxB['x'])
    yA = max(boxA['y'], boxB['y'])
    xB = min(boxA['x'] + boxA['width'], boxB['x'] + boxB['width'])
    yB = min(boxA['y'] + boxA['height'], boxB['y'] + boxB['height'])

    # Calculate area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Calculate area of both bounding boxes
    boxAArea = boxA['width'] * boxA['height']
    boxBArea = boxB['width'] * boxB['height']

    # Calculate IoU
    if float(boxAArea + boxBArea - interArea) != 0: # Prevent divide by 0
      iou = interArea / float(boxAArea + boxBArea - interArea)
    else:
      iou = 0

    return iou

# Match predicted tracks to ground truth tracks

In [ ]:
def match_predictions_gt(alpha=0.5,results={}):
  # Match is when the iou>alpha
  alpha = 0.5
  TPTr = {k: v for k, v in results.items() if v["iou"] >= alpha}
  PrTrajs = dict(sorted(results.items(), key=lambda x: x[1]["iou"], reverse=True))
  # [print(sorted_results[item]) for item in results]
  # check last item in results because it is a list with None objects
  return TPTr,PrTrajs

# Calculate precision and recall values

In [ ]:
def calc_prec_recall(PrTrajs,results,alpha=0.5):
  n = 1
  TPTrn = 0
  for item in PrTrajs:
    if PrTrajs[item]["iou"] > alpha:
      TPTrn += 1
    PrTrajs[item]["Precision"] = TPTrn/n
    PrTrajs[item]["Recall"] = TPTrn/len(results)
    n+=1
    # print(TPTrn,n)
  return PrTrajs

# Calculate IDF1

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def compute_idf1(pred_boxes, gt_boxes):
    num_pred = len(pred_boxes)
    num_gt = len(gt_boxes)

    iou_matrix = np.zeros((num_pred, num_gt))

    for i, pred_box in enumerate(pred_boxes):
        for j, gt_box in enumerate(gt_boxes):
            pred = 0
            if gt_box["x"] != -1:
                pred_iou = calculate_iou(pred_box,gt_box)
                if pred_iou > pred:
                  pred = pred_iou
              # only one mouse [-1,-1]
            else:
                pred = 1
            iou_matrix[i, j] = pred

    row_ind, col_ind = linear_sum_assignment(-iou_matrix)

    tp = 0
    for i, j in zip(row_ind, col_ind):
        if iou_matrix[i, j] >= 0.5:
            tp += 1

    fn = num_gt - tp
    fp = num_pred - tp

    id_precision = tp / (tp + fp) if tp + fp > 0 else 0
    id_recall = tp / (tp + fn) if tp + fn > 0 else 0
    id_f1 = 2 * id_precision * id_recall / (id_precision + id_recall) if id_precision + id_recall > 0 else 0

    return id_f1

def calculate_idf1(pred_path, gt_path):

    with open(pred_path, 'r') as f:
        # Load the JSON data into a dictionary
        pred_annotations = json.load(f)

    with open(gt_path, 'r') as f:
        # Load the JSON data into a dictionary
        gt_annotations = json.load(f)

    idf1_sum = 0
    num_frames = 0

    for image in pred_annotations:
        gt_image = find_gt_annotation(gt_annotations, image)
        if gt_image != None:
            num_frames += 1
            pred_boxes = get_pred_image_boxes(image)
            gt_boxes = [item for item in gt_image["annotations"] if len(item) > 3]

            idf1 = compute_idf1(pred_boxes, gt_boxes)
            idf1_sum += idf1

    idf1_avg = idf1_sum / num_frames
    print(f"The average ID-F1 score is: {idf1_avg}")
    return idf1_avg


In [ ]:
def get_file(string):
  terms = string.split("/")[-3:]
  joined_terms = "/".join(terms)
  return joined_terms[:-5]

In [ ]:
import numpy as np
path = '/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/'

directories = [dir for dir in os.listdir(path) if os.path.isdir(os.path.join(path, dir))]
pred_path = "/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/topview/alphapose-results.json"
evaluation = {}
for file in os.listdir(path+"/"+directories[2]):
  if "top" in file:
    gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/one_mouse_top.json"
    pred_path = path+directories[2]+"/"+file+"/alphapose-results.json"
    print(pred_path)
    evaluation[get_file(pred_path)] = calculate_idf1(pred_path,gt_path)
  elif "side" in file:
    gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/one_mouse_side.json"
    pred_path = path+directories[2]+"/"+file+"/alphapose-results.json"
    print(pred_path)
    evaluation[get_file(pred_path)] = calculate_idf1(pred_path,gt_path)
  elif "face" in file:
    gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/one_mouse_face.json"
    pred_path = path+directories[2]+"/"+file+"/alphapose-results.json"
    print(pred_path)
    evaluation[get_file(pred_path)] = calculate_idf1(pred_path,gt_path)


/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/sideview/alphapose-results.json
The average ID-F1 score is: 0.5564780617521057
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/faceview_ckpt2/alphapose-results.json
The average ID-F1 score is: 0.5610749094959611
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/faceview/alphapose-results.json
The average ID-F1 score is: 0.4009344229861915
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/sideview_ckpt2/alphapose-results.json
The average ID-F1 score is: 0.5964935537533775
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/sideview_ckpt1/alphapose-results.json
The average ID-F1 score is: 0.6890190336749681
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/faceview_ckpt1/alphapose-results.json
The average ID-F1 score is: 0.666915422885576

In [ ]:
for directory in directories:
  for file in os.listdir(path+"/"+directory):
    if "top" in file:
      gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_top.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_idf1(pred_path,gt_path)
    elif "side" in file:
      gt_path = "//content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_side.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_idf1(pred_path,gt_path)
    elif "face" in file:
      gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_face.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_idf1(pred_path,gt_path)

/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/sideview_ckpt1/alphapose-results.json
The average ID-F1 score is: 0.3647485950902087
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/faceview_ckpt2/alphapose-results.json
The average ID-F1 score is: 0.2860812119643636
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/faceview_ckpt1/alphapose-results.json
The average ID-F1 score is: 0.35983495289168443
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/faceview/alphapose-results.json
The average ID-F1 score is: 0.1243220899470904
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/sideview/alphapose-results.json
The average ID-F1 score is: 0.16505801687763713
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_o

In [ ]:

# [print(evaluation) for evaluation in evaluation.items()]
path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/evaluation_IDF1.json"

evaluations_df = pd.DataFrame(evaluation.items(),columns = ["File","IDF1"])
with open(path, "w") as outfile:
    json.dump(evaluation, outfile)